In [41]:
     
class MinHeap:
    
    def __init__(self):
        self.nodes = [0]
        
    def addNode(self, vertex):
        self.nodes.append(vertex)
        self.heapifyUp(len(self.nodes) - 1)
    
    def makeHeap(self):
        count = len(self.nodes) - 1
        for i in range(count, 1, -1):
            self.heapifyUp(i)
                 
    def heapifyUp(self, index):
        if index == 1:
            return
        if self.nodes[index].getEstimatedDistance() < self.nodes[index//2].getEstimatedDistance():
            self.swapValue(index, index//2)
            self.heapifyUp(index//2)
        else:
            return
            
        
    def heapifyDown(self, index):
        if self.isOutOfBound(index):
            return
        if self.isOutOfBound(2 * index):
            return
        if self.isOutOfBound(2 * index + 1):
            j = 2 * index
        else:
            if self.nodes[2 * index + 1].getEstimatedDistance() < self.nodes[2 * index].getEstimatedDistance():
                j = 2 * index + 1
            else:
                j = 2 * index
        if self.nodes[index].getEstimatedDistance() < self.nodes[j].getEstimatedDistance():
            return
        self.swapValue(index, j)
        self.heapifyDown(j)
            
        
    def retrieveMin(self):
        if len(self.nodes) > 0:
            low = self.nodes[1]
            self.swapValue(1, len(self.nodes) - 1)
            del self.nodes[-1]
            self.heapifyDown(1)
            return low
        else:
            return None
        
    def getMin(self):
        if len(self.nodes) > 0:
            return self.nodes[1]
        return None
    
    def swapValue(self, pos1, pos2):
        if not (self.isOutOfBound(pos1) and self.isOutOfBound(pos2)):
            temp = self.nodes[pos1]
            self.nodes[pos1] = self.nodes[pos2]
            self.nodes[pos2] = temp
    
    def inHeap(self, vertex):
        if vertex in self.nodes:
            return self.nodes.index(vertex)
        else:
            return None
        
    def isEmpty(self):
        return len(self.nodes) == 1
    
    def isOutOfBound(self, index):
        if index >= len(self.nodes) or index < 1:
            return True
        else:
            return False
        
    def output(self):
        for i in range(1, len(self.nodes)):
            print(self.nodes[i].getLabel() + "," + str(self.nodes[i].getEstimatedDistance()), end = " ")
        print()
           
    
import math

class Vertex:
    
    def __init__(self, label):
        self.label = label
        self.neighbors = {}
        self.parent = []
        self.estimatedDist = 0
        self.location = (0,0)
        self.pathNumber = 0
    
    def getLabel(self):
        return self.label
    
    def getNeighbors(self):
        return self.neighbors
    
    def addEdge(self, weight, vertex):
        if weight < 0:
            return
        if vertex not in self.neighbors:
            self.neighbors[vertex] = weight
            vertex.neighbors[self] = weight
    
    def getParent(self):
        return self.parent
    
    def resetParent(self):
        self.parent.clear()
    
    def addParent(self, parent):
        self.parent.append(parent)
        
    def getEstimatedDistance(self):
        return self.estimatedDist
    
    def setEstimatedDistance(self, value):
        self.estimatedDist = value
        
    def getEdgeWeight(self, vertex):
        if vertex not in self.neighbors:
            return 0
        else:
            return self.neighbors[vertex]
        
    def getLocation(self):
        return self.location
    
    def setLocation(self, coordinate):
        self.location = coordinate
        
    def getPathNumber(self):
        return self.pathNumber
    
    def setPathNumber(self, value):
        self.pathNumber = value
        
    def getDistance(self, vertex):
        coor1 = self.location
        coor2 = vertex.getLocation()
        return math.sqrt(pow(coor1[0]-coor2[0],2) + pow(coor1[1]-coor2[1],2))

class CityPlan:
    
    def __init__(self):
        self.graph = []
        self.edgeNum = 0
    
    def getGraph(self):
        return self.graph
    
    def getVertexIndex(self, label):
        for index in range(len(self.graph)):
            if self.graph[index].getLabel() == label:
                return index
        return None
            
    def readInput(self, fileName):
        with open(fileName, 'r') as file:
            transition = False
            count = 0
            for lines in file:
                lines = lines.split()
                if transition == False:
                    if len(lines) == 0:
                        transition = True
                        continue
                    elif len(lines) == 2:
                        for number in range(int(lines[0])):
                            self.graph.append(Vertex(number))
                        self.edgeNum = int(lines[1])
                    elif len(lines) == 3:
                        label = int(lines[0])
                        index = self.getVertexIndex(label)
                        self.graph[index].setLocation((int(lines[1]), int(lines[2])))
                else:
                    count += 1
                    if count > self.edgeNum:
                        break
                    start = self.graph[self.getVertexIndex(int(lines[0]))]
                    end = self.graph[self.getVertexIndex(int(lines[1]))]
                    distance = start.getDistance(end)
                    start.addEdge(distance, end)
    
    def Dijkstra(self, sourceVertex):
        for vertices in self.graph:
            if vertices.getLabel() == sourceVertex:
                vertices.setEstimatedDistance(0)
                vertices.setPathNumber(1)
            else:
                vertices.setEstimatedDistance(math.inf)
        minHeap = MinHeap()        
        for element in self.graph:
            minHeap.addNode(element)
        while not minHeap.isEmpty():
            top = minHeap.retrieveMin()
            adjacent = top.getNeighbors()
            for neighbor in adjacent:
                if minHeap.inHeap(neighbor) != None:
                    if neighbor.getEstimatedDistance() > top.getEstimatedDistance() + top.getEdgeWeight(neighbor):
                        neighbor.setEstimatedDistance(top.getEstimatedDistance() + top.getEdgeWeight(neighbor))
                        minHeap.heapifyUp(minHeap.inHeap(neighbor))
                        neighbor.resetParent()
                        neighbor.addParent(top)
                        neighbor.setPathNumber(top.getPathNumber())
                    elif neighbor.getEstimatedDistance() == top.getEstimatedDistance() + top.getEdgeWeight(neighbor):
                        neighbor.addParent(top)
                        neighbor.setPathNumber(neighbor.getPathNumber() + top.getPathNumber())
                 
    def DijkstraVariant(self, sourceVertex, endVertex, A, B):
        variant = 0
        end = self.graph[self.getVertexIndex(endVertex)]
        for vertices in self.graph:
            if vertices.getLabel() == sourceVertex:
                vertices.setEstimatedDistance(0)
                vertices.setPathNumber(1)
            else:
                vertices.setEstimatedDistance(math.inf)
        minHeap = MinHeap()        
        for element in self.graph:
            minHeap.addNode(element)
        while not minHeap.isEmpty():
            top = minHeap.retrieveMin()
            adjacent = top.getNeighbors()
            for neighbor in adjacent:
                if minHeap.inHeap(neighbor) != None:
                    variant = (A * (top.getEstimatedDistance() + top.getEdgeWeight(neighbor))) + (B * (neighbor.getDistance(end) - top.getDistance(end)))
                    if neighbor.getEstimatedDistance() > variant:
                        neighbor.setEstimatedDistance(variant)
                        minHeap.heapifyUp(minHeap.inHeap(neighbor))
                        neighbor.resetParent()
                        neighbor.addParent(top)
                        neighbor.setPathNumber(top.getPathNumber())
                    elif neighbor.getEstimatedDistance() == variant:
                        neighbor.addParent(top)
                        neighbor.setPathNumber(neighbor.getPathNumber() + top.getPathNumber())
                        
    def shortestPathDistance(self, sourceLabel):
        self.Dijkstra(sourceLabel)
        shortestDistance = []
        for vertex in self.graph:
            shortestDistance.append(round(vertex.getEstimatedDistance()))
        return shortestDistance
    
    def addNode(self, label):
        self.graph.append(Vertex(label))
        
    def addEdges(self, source, end, weight):
        self.graph[self.getVertexIndex(source)].addEdge(weight, self.graph[self.getVertexIndex(end)])
        
    def noOfShortestPaths(self, start, end):
        self.Dijkstra(start)
        return self.graph[self.getVertexIndex(end)].getPathNumber()
    
    def fromSrcToDest(self, source, end, A, B):
        self.DijkstraVariant(source, end, A, B)
        path = []
        start = self.graph[self.getVertexIndex(source)]
        end = self.graph[self.getVertexIndex(end)]
        path.append(end.getLabel())
        while end != start:
            path.append(end.getParent()[0].getLabel())
            end = end.getParent()[0]
        path.reverse()
        return path
    
    def fromSrcToDestVia(self, source, end, stops, A, B):
        if len(stops) == 0 or stops == None:
            return self.fromSrcToDest(source, end, A, B)
        start = source
        dest = stops.pop(0)
        path = self.fromSrcToDest(start, dest, A, B)
        for i in range(len(stops)):
            start = dest
            dest = stops.pop(0)
            path += self.fromSrcToDest(start, dest, A, B)[1::]
        start = dest
        return path + self.fromSrcToDest(start, end, A, B)[1::]
            
    
    def primAlgo(self, source):
        for vertices in self.graph:
            if vertices.getLabel() == source:
                vertices.setEstimatedDistance(0)
                vertices.addParent(vertices)
            else:
                vertices.setEstimatedDistance(math.inf)
        minHeap = MinHeap()        
        for element in self.graph:
            minHeap.addNode(element)
        while not minHeap.isEmpty():
            top = minHeap.retrieveMin()
            adjacent = top.getNeighbors()
            for neighbor in adjacent:
                if minHeap.inHeap(neighbor) != None:
                    if neighbor.getEstimatedDistance() > top.getEdgeWeight(neighbor):
                        neighbor.setEstimatedDistance(top.getEdgeWeight(neighbor))
                        neighbor.resetParent()
                        neighbor.addParent(top)
                        minHeap.heapifyUp(minHeap.inHeap(neighbor))
                        
    def minCostReachabilityFromSrc(self, source):
        self.primAlgo(source)
        array = []
        for element in self.graph:
            if len(element.getParent()) == 0:
                array.append(-1)
            else:
                array.append(element.getParent()[0].getLabel())
        return array
        
    def minCostOfReachabilityFromSrc(self, source):
        self.primAlgo(source)
        totalMinCost = 0
        for element in self.graph:
            if len(element.getParent()) != 0:
                parent = element.getParent()[0]
                totalMinCost += element.getEdgeWeight(parent)
        return round(totalMinCost)
        
    def isFullReachableFromSrc(self, source):
        reachabilityTree = self.minCostReachabilityFromSrc(source)
        if -1 in reachabilityTree:
            return False
        else:
            return True
        

In [92]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
print(plan.fromSrcToDest(1,8,1,0))
"""
for element in plan.getGraph():
    print(str(element.getLabel()) + ": " + str(element.getEstimatedDistance()))
    p = element
    print("\t", end = "")
    while p != None:
        print(p.getLabel(), end = " ")
        p = p.getParent()
    print()
"""

[1, 0, 5, 7, 8]


'\nfor element in plan.getGraph():\n    print(str(element.getLabel()) + ": " + str(element.getEstimatedDistance()))\n    p = element\n    print("\t", end = "")\n    while p != None:\n        print(p.getLabel(), end = " ")\n        p = p.getParent()\n    print()\n'

In [13]:
plan = CityPlan()
plan.readInput("CityPlan0.txt")
print(plan.fromSrcToDest(0,3,0,1))

[0, 4, 3]


In [82]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
print(plan.shortestPathDistance(3))

[38.2842712474619, 48.2842712474619, 10.0, 0, 14.142135623730951, 24.14213562373095, 29.14213562373095, 80.62257748298549, 85.62257748298549, inf]


In [16]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
print(plan.noOfShortestPaths(3,6))

2


In [18]:
plan = CityPlan()
plan.addNode(0)
plan.addNode(1)
plan.addNode(2)
plan.addNode(3)
plan.addNode(4)
plan.addNode(5)
plan.addNode(6)
plan.addEdges(0,1,1)
plan.addEdges(0,2,1)
plan.addEdges(1,3,1)
plan.addEdges(2,3,1)
plan.addEdges(3,4,1)
plan.addEdges(3,5,1)
plan.addEdges(4,6,1)
plan.addEdges(5,6,1)
print(plan.noOfShortestPaths(0,6))

4


In [19]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
print(plan.minCostReachabilityFromSrc(3))

[5, 0, 3, 3, 2, 4, 5, 4, 7, -1]


In [20]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
print(plan.minCostOfReachabilityFromSrc(3))

134.14213562373095


In [21]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
print(plan.isFullReachableFromSrc(3))

False


In [22]:
plan = CityPlan()
plan.readInput("CityPlan1.txt")
stops = [5,7,2]
print(plan.fromSrcToDestVia(3,8,stops,1,0))

[3, 2, 5, 7, 4, 2, 4, 7, 8]


In [47]:
plan = CityPlan()
plan.readInput("CityPlan4.txt")
print(plan.fromSrcToDest(0, 210, 1, 0))
print(plan.noOfShortestPaths(0,700))
print(plan.noOfShortestPaths(0,39998))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 210]
176851
1
